# Entity Explorer - Account
 <details>
     <summary>&nbsp;<u>Notebook Details...</u></summary>

 **Notebook Version:** 2.0<br>
 **Python Version:** Python 3.8+<br>
 **Required Packages**: msticpy, msticnb<br>

 **Data Sources Required**:
 - Sentinel - SecurityAlert, SecurityEvent, HuntingBookmark, Syslog, AAD SigninLogs, AzureActivity, OfficeActivity, ThreatIndicator
 - (Optional) - VirusTotal, AlienVault OTX, IBM XForce, Open Page Rank, (all require accounts and API keys)
 </details>

Brings together a series of queries and visualizations to help you determine the security state of an Account. 

The account can be a Windows or Linux account or an Azure Active Directory/Office 365 account.

The notebook uses the [MSTIC notebooklets](https://msticnb.readthedocs.io) package to run most of the functionality.
Summarized data is returned when it is run and more detailed information is contained in the returned `result` class.

<!DOCTYPE html>
<html>
  <head>
  </head>
  <body>
    <h1>Contents<span class="tocSkip"></span></h1>
    <div class="toc">
      <ul class="toc-item">
        <li><span><a href="#Hunting-Hypothesis">Hunting Hypothesis</a></span></li>
        <li><span><a href="#Notebook-initialization">Notebook initialization</a></span></li>
        <li><span><a href="#Enter-account-name-and-query-time">Enter account
              name and query time</a></span></li>
        <li><span><a href="#Run-the-main-account_summary-function">Run the main
              account_summary function</a></span></li>
        <li><span><a href="#Retrieve-additional-data-for-the-selected-account">Retrieve
              additional data for the selected account</a></span></li>
        <li><span><a href="#Browse-alerts">Browse alerts</a></span></li>
        <li><span><a href="#Browse-other-event-data">Browse other event data</a></span></li>
        <li><span><a href="#More-operations-available">
              Using additional properties and methods of the result object<br>
            </a></span></li>
        <li><span><a href="#More-information">More information</a></span></li>
      </ul>
    </div>
    
  </body>
</html>


## Hunting Hypothesis
Our broad initial hunting hypothesis is that a we have received account name entity which is suspected to be compromised and is being used malicious manner in internal networks, we will need to hunt from a range of different positions to validate or disprove this hypothesis.

---
# Notebook initialization
This should complete without errors. If you encounter errors or warnings look at the following notebooks:

- <a href="https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A%20Getting%20Started%20Guide%20For%20Azure%20Sentinel%20ML%20Notebooks.ipynb">Getting Started Notebook</a>
- [TroubleShootingNotebooks](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/TroubleShootingNotebooks.ipynb)
- [ConfiguringNotebookEnvironment](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)

<details>
    <summary>&nbsp;<u>Details...</u></summary>
The next cell:
- Checks for the correct Python version
- Checks versions and optionally installs required packages
- Imports the required packages into the notebook
- Sets a number of configuration options.

If you are running in the Azure Sentinel Notebooks environment (Azure Notebooks or Azure ML) you can run live versions of these notebooks:
- [Getting Started](./A Getting Started Guide For Azure Sentinel ML Notebooks.ipynb)
- [Run TroubleShootingNotebooks](./TroubleShootingNotebooks.ipynb)
- [Run ConfiguringNotebookEnvironment](./ConfiguringNotebookEnvironment.ipynb)

You may also need to do some additional configuration to successfully use functions such as Threat Intelligence service lookup and Geo IP lookup. 
There are more details about this in the `ConfiguringNotebookEnvironment` notebook and in these documents:
- [msticpy configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
- [Threat intelligence provider configuration](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html#configuration-file)
</details>

In [1]:
from datetime import datetime, timedelta, timezone

REQ_PYTHON_VER = "3.8"
REQ_MSTICPY_VER = "1.8.0"


# %pip install --upgrade msticpy

import msticpy
msticpy.init_notebook(
    namespace=globals(),
    additional_packages=["msticnb>=1.0"],
    verbosity=0,
);


In [2]:
# papermill default parameters
ws_name = "Default"
account_name = ""
account_types = ["All"]  # Windows, Linux, Azure, All
end = datetime.now(timezone.utc)
start = end - timedelta(days=2)


### Get Workspace and Authenticate

<details>
    <summary><u>Authentication help...</u></summary>
    If you want to use a workspace other than one you have defined in your<br>
msticpyconfig.yaml create a connection string with your AAD TENANT_ID and<br>
your WORKSPACE_ID (these should both be quoted UUID strings).

```python
  workspace_cs = "loganalytics://code().tenant('TENANT_ID').workspace('WORKSPACE_ID')"
```
e.g.
```python
  workspace_cs = "loganalytics://code().tenant('c3de0f06-dcb8-40fb-9d1a-b62faea29d9d').workspace('c62d3dc5-11e6-4e29-aa67-eac88d5e6cf6')"
```
Then in the Authentication cell replace
the call to `qry_prov.connect` with the following:
```python
  qry_prov.connect(connect_str=workspace_cs)
```
The cell should now look like this:

```python
...
  # Authentication
  qry_prov = QueryProvider(data_environment="MSSentinel")
  qry_prov.connect(connect_str=workspace_cs)
...
```

On successful authentication you should see a ```popup schema``` button.
To find your Workspace Id go to [Log Analytics](https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.
</details>

In [3]:
print("Configured workspaces: ", ", ".join(msticpy.settings.get_config("AzureSentinel.Workspaces").keys()))
import ipywidgets as widgets
ws_param = widgets.Combobox(
    description="Workspace Name",
    value=ws_name,
    options=list(msticpy.settings.get_config("AzureSentinel.Workspaces").keys())
)
ws_param

Configured workspaces:  Default


Combobox(value='Default', description='Workspace Name', options=('Default',))

In [4]:
from msticpy.common.timespan import TimeSpan

# Authentication
qry_prov = QueryProvider(data_environment="MSSentinel")
qry_prov.connect(WorkspaceConfig(workspace=ws_param.value))

nb_timespan = TimeSpan(start, end)
qry_prov.query_time.timespan = nb_timespan
md("<hr>")
md("Confirm time range to search", "bold")
qry_prov.query_time

try_azcli_login=True;enable_add_items_to_help=False
{'try_azcli_login': 'True', 'enable_add_items_to_help': 'False'}
try_azcli_login=True;enable_add_items_to_help=False
Connecting... 

connected


#### Authentication and Configuration problems?

If you are having problems, expand the details section below
<br>
<details>
    <summary>Click for details about configuring your authentication parameters</summary>
    
    
The notebook is expecting your Azure Sentinel Tenant ID and Workspace ID to be configured in one of the following places:
- `msticpyconfig.yaml` in the current folder or location specified by `MSTICPYCONFIG` environment variable.
- `config.json` in the current folder
    
For help with setting up your configuration (if this hasn't been done automatically) see the [Getting Started](./A Getting Started Guide For Azure Sentinel ML Notebooks.ipynb) notebook in the root folder of your Azure-Sentinel-Notebooks project.
</details>

## Import and initialize notebooklets

This imports the **msticnb** package and the notebooklets classes.

These are needed for the notebook operations

In [5]:
import msticnb as nb

nb.init(query_provider=qry_prov)
pivot.timespan = qry_prov.query_time.timespan

Notebooklets: 11 notebooklets loaded.
try_azcli_login=True;enable_add_items_to_help=False
{'try_azcli_login': 'True', 'enable_add_items_to_help': 'False'}
try_azcli_login=True;enable_add_items_to_help=False
Notebooklets: Loaded providers: MSSentinel, geolitelookup, tilookup


# Enter account name and query time window
Type the account name that you want to search for and the time bounds over which you want to search. 

You can specify the account as:

- a simple user name (e.g. `alice`)
- a user principal name (`alice@contoso.com`)
- a qualified windows user name `mydomain\alice`

In the second two cases the domain qualifier will be stripped off before the search. The search is not case sensitive and will match full substrings. E.g. `bob` will match `domain\bob` and `bob@contoso.com` but not `bobg` or `bo`.

In [6]:
account_txt = nbwidgets.GetText(prompt='Enter the Account name to search for:', value=account_name)

display(account_txt)
md("<hr>")

Text(value='', description='Enter the Account name to search for:', layout=Layout(width='50%'), style=Descript…

You can opt to search all data types or just a subset.
For example, if you know the account activity that you are interested in is only Windows host activity, you can select "Windows".

The default is All Data but other options are:
- Azure (includes Office365 activity, Active Directory Signin logs and Azure audit activity)
- Office365 - only Office365 activity
- AzureActiveDirectory - only signin logs
- Azure - only Azure resource/audit activity
- Windows - only Windows host event logs
- Linux - only Linux host syslog

In [7]:
from msticnb.nb.azsent.account.account_summary import AccountType
acct_types = [item.name for item in AccountType]
import ipywidgets as widgets
acct_types_select = widgets.SelectMultiple(
    description="Select Account types to search",
    options=acct_types,
    value=account_types or acct_types,
    **WIDGET_DEFAULTS,
)
acct_types_select


SelectMultiple(description='Select Account types to search', index=(6,), layout=Layout(width='95%'), options=(…

# Run the main *account_summary* notebooklet `run` method

The notebooklet will search Azure, Windows host and Linux host data,
searching for account matches.

**Note**: Different result properties will populated for different account types.

It will display a *summary* of the information retrieved as it is running.
You can find information on accessing the full data later in the notebook.

**Note:** If more than one matching account name is found, all matches will be shown.<br>
You can select each of these matching accounts to view more details about the account.<br>
Once selected, you can retrieve more detailed information about that account.

<details>
<summary><b>Running the notebooklet as a Pivot Function</b></summary>
This can also be run as a pivot function from the Account entity.<br>
The pivot function `account_summary` is in the `nblt` container
of the `Account` entity.

```python
    Account = entities.Account
    acc_result = Account.nblt.account_summary(
        value=account_txt.value,
        account_types=acct_types_select.value
    )
```

In this case you do not have direct access to the methods
of the "AccountSummary".<br>
However, all methods and properties
of the notebooklet class are accessible via the results class.
```python
   acct_nb.get_additional_data()
```
is equivalent to
```python
   acc_result.get_additional_data()
```
</details>

In [8]:
acct_nb = nb.nblts.azsent.account.AccountSummary()

md(
    "",
    "large, bold"
    )
acc_result = acct_nb.run(
    value=account_txt.value,
    timespan=qry_prov.query_time.timespan,
    account_types=acct_types_select.value,
)

No local Maxmind City Database found.  Attempting to downloading new database to C:\Users\gcass\.msticpy
Extraction complete. Local Maxmind city DB: C:\Users\gcass\.msticpy\GeoLite2-City.mmdb.16988.tar.gz


Account(Name=griffin.cass@wedc.org, UPNSuffix=wedc.org, AadTenantId=a4d0c603-9c7f-4293-b...)

,TimeGenerated,TenantId,SourceSystem,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationContextClassReferences,AuthenticationDetails,AppliedEventListeners,AuthenticationMethodsUsed,...,Status,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,UserType,FlaggedForReview,IPAddressFromResourceProvider,SignInIdentifier,SignInIdentifierType,ResourceTenantId,HomeTenantId,UniqueTokenIdentifier,SessionLifetimePolicies,AutonomousSystemNumber,AuthenticationProtocol,CrossTenantAccessType,AppliedConditionalAccessPolicies,RiskLevel,Type,AccountName,Source
0,2023-06-23 19:54:39.132402200+00:00,a97557b5-b341-450d-ac71-cdd5658a7e03,Azure AD,/tenants/a4d0c603-9c7f-4293-b89c-a08c1a8eb571/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,afce81f3-0eab-427f-9151-5effe253ab90,Microsoft.aadiam,Microsoft.aadiam,,Griffin Cass,4,US,,Office365 Shell WCSS-Client,89bee1f7-5e6e-4d8a-9f3d-ecd601259da7,"[{""id"":""c1"",""detail"":""previouslySatisfied""}]","[{""authenticationStepDateTime"":""2023-06-23T19:52:51.7802923+00:00"",""authenticationMethod"":""Previ...",None,,...,"{'errorCode': 0, 'additionalDetails': 'MFA requirement satisfied by claim in the token'}",,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0....",Griffin Cass,36a4c34a-dd20-4357-b32c-77007b86bbc4,griffin.cass@wedc.org,a4d0c603-9c7f-4293-b89c-a08c1a8eb571,Member,None,,,,a4d0c603-9c7f-4293-b89c-a08c1a8eb571,a4d0c603-9c7f-4293-b89c-a08c1a8eb571,bT8Q965qPkG1nSJAOuBQAA,[],4181,none,none,,,SigninLogs,griffin.cass@wedc.org,AzureActiveDirectory


In [9]:
if len(acc_result.account_selector.options) > 1:
    md_warn("Multiple matches found for account. Running on first listed account")

# Retrieve additional data for the selected account

The result returned from the last cell has a number of properties and methods that you can use
to retrieve and view further information.

The main one for this notebooklet is `get_additional_data`.
Depending on the account type (Azure, Windows or Linux), it will retrieve more detailed
data about recent activity

In [10]:
acc_result.get_additional_data()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

OperationCount  \
UserPrincipalName                  Type           IPAddress             AppResourceProvider                          UserType                   
griffin.cass@0sdss.onmicrosoft.com SigninLogs     69.130.255.13         Inform Development                           Guest                  1   
                                                                        My Apps                                      Guest                  5   
                                                                        Salesforce                                   Guest                  1   
                                                                        Salesforce Partial                           Guest                  1   
griffin.cass@wedc.org              OfficeActivity 144.92.154.82         Exchange                                     Regular                2   
                                                  192.168.1.72          MicrosoftTeams                               Regular                1   
                                                  192.168.86.42         MicrosoftTeams                               Regular                2   
                                                  20.190.157.160        Exchange                                     Regular               14   
                                                  20.190.157.27         Exchange                                     Regular               11   
                                                  20.190.157.30         Exchange                                     Regular               22   
                                                  20.190.157.96         Exchange                                     Regular               18   
                                                  20.190.157.97         Exchange                                     Regular               20   
                                                  216.56.240.114        OneDrive                                     Regular                1   
                                                  2600:387:c:5516::3    Exchange                                     Regular               12   
                                                  2603:10b6:208:23d::15 Exchange                                     Regular                1   
                                                  2603:10b6:208:3ac::10 Exchange                                     Regular                1   
                                                  2603:10b6:208:3da::13 Exchange                                     Regular               20   
                                                  2603:10b6:208:481::10 Exchange                                     Regular               12   
                                                  2603:10b6:208:48a::21 Exchange                                     Regular               12   
                                                  2603:10b6:208:48c::19 Exchange                                     Regular               11   
                                                  40.126.29.96          Exchange                                     Regular               22   
                                                  52.104.99.160         OneDrive                                     Regular                3   
                                                  52.104.99.161         OneDrive                                     Regular                2   
                                                  52.104.99.164         OneDrive                                     Regular                1   
                                                  52.104.99.165         OneDrive                                     Regular                1   
                                                  52.104.99.166         OneDrive                                     Regular                2   
                                                  52.104.99.169    

Count  \
Source               Operation              UserAgent                                                                                            IpAddress             AsnDescription                  CountryCode          
AzureActiveDirectory Sign-in activity       Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0... 69.130.255.13         TDS-AS, US                      US             105   
                                            Windows-AzureAD-Authentication-Provider/1.0                                                          216.56.240.114        WISCNET1-AS, US                 US               2   
                                                                                                                                                 69.130.255.13         TDS-AS, US                      US               3   
Office365            AddedToGroup           Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0... 69.130.255.13         TDS-AS, US                      US               2   
                     CompanyLinkUsed        Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0... 69.130.255.13         TDS-AS, US                      US               2   
                     Create                                                                                                                      2603:10b6:208:481::10 NA                              US               3   
                                                                                                                                                 69.130.255.13         TDS-AS, US                      US              23   
                     FileAccessed           Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0... 69.130.255.13         TDS-AS, US                      US               5   
                                            OneDriveMpc-Transform_Thumbnail/1.0                                                                  52.104.99.174         MICROSOFT-CORP-MSN-AS-BLOCK, US US               1   
                     FileModified           Microsoft SkyDriveSync 23.114.0530.0001 ship; Windows NT 10.0 (19042)                                69.130.255.13         TDS-AS, US                      US               1   
                     FilePreviewed          Microsoft SkyDriveSync 23.114.0530.0001 ship; Windows NT 10.0 (19042)                                69.130.255.13         TDS-AS, US                      US               2   
                                            OneDriveMpc-Transform_Thumbnail/1.0                                                                  52.104.99.160         MICROSOFT-CORP-MSN-AS-BLOCK, US US               3   
                                                                                                                                                 52.104.99.161         MICROSOFT-CORP-MSN-AS-BLOCK, US US               2   
                                                                                                                                                 52.104.99.164         MICROSOFT-CORP-MSN-AS-BLOCK, US US               1   
                                                                                                                                                 52.104.99.165         MICROSOFT-CORP-MSN-AS-BLOCK, US US               1   
                                                                                                                                                 52.104.99.166         MICROSOFT-CORP-MSN-AS-BLOCK, US US               2   
                                                                                                                                                 52.104.99.169         MICROSOFT-CORP-MSN-AS-BLOCK, US US               1   
             

# Browse alerts
If there are any alerts referencing this account name they can be viewed
by calling the `acc_result.browse_alerts()` function.

In [11]:
acc_result.notebooklet.browse_alerts()


No data is available for last_result.related_alerts.


# Appendix - Additional properties from the Notebooklet result

---

## Browse other event data

You can use a simple view to make it easier to see the details of individual events
by calling the "view_events" method.

You need to supply the name of the result attribute that you want to view plus
one or more summary columns (as a list of strings).

In [12]:
data_source = nbwidgets.SelectItem(
    description="Available data properties\n",
    item_list=acc_result.data_properties()
)
data_source

Running the following cell will use the data property selected above
to browse through the data (if any).

In [13]:
acc_result.view_events(
    attrib=data_source.value,  # the result attribute to view
    # summary_cols controls which cols are use to create the summary
    # for the select list
    summary_cols = list(getattr(acc_result, data_source.value).columns)[:3]
    # Add specific columns here to customize the list items
    # summary_cols=["SourceSystem", "Operation", "IPAddress"]
)

,0
TimeGenerated,2023-06-23 19:54:39.132402200+00:00
TenantId,a97557b5-b341-450d-ac71-cdd5658a7e03
SourceSystem,Azure AD
ResourceId,/tenants/a4d0c603-9c7f-4293-b89c-a08c1a8eb571/providers/Microsoft.aadiam
OperationName,Sign-in activity
OperationVersion,1.0
Category,SignInLogs
ResultType,0
ResultSignature,None
DurationMs,0


You can also access the DataFrames properties directly

In [14]:
acc_result.ip_all_data

,IPAddress_orig,IPAddress_x,CountryCode,CountryName,State,City,Longitude,Latitude,IpAddress,asn,query,asn_cidr,asn_country_code,asn_registry,asn_date,asn_description,nets,Whois_data,AsnDescription,TenantId,SourceSystem,TimeGenerated,ResourceId,Operation,OperationVersion,...,ExtraProperties,AddOnType,AddonName,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,ClientAppId,ApplicationId,SRPolicyId,SRPolicyName,SRRuleMatchDetails,IsManagedDevice,ActorContextId_
0,69.130.255.13,69.130.255.13,US,United States,Wisconsin,Middleton,-89.5073,43.103,69.130.255.13,4181,69.130.255.13,69.130.240.0/20,US,arin,2003-03-25,"TDS-AS, US","[{'cidr': '69.128.0.0/14', 'handle': 'NET-69-128-0-0-1', 'name': 'NETBLK-TDSNET-BLK', 'startAddr...","{'asn': '4181', 'query': '69.130.255.13', 'asn_cidr': '69.130.240.0/20', 'asn_country_code': 'US...","TDS-AS, US",a97557b5-b341-450d-ac71-cdd5658a7e03,OfficeActivityManager,2023-06-21 20:21:55+00:00,,MailItemsAccessed,NaN,...,None,,,,,,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",,,,,None,None,
1,69.130.255.13,69.130.255.13,US,United States,Wisconsin,Middleton,-89.5073,43.103,69.130.255.13,4181,69.130.255.13,69.130.240.0/20,US,arin,2003-03-25,"TDS-AS, US","[{'cidr': '69.128.0.0/14', 'handle': 'NET-69-128-0-0-1', 'name': 'NETBLK-TDSNET-BLK', 'startAddr...","{'asn': '4181', 'query': '69.130.255.13', 'asn_cidr': '69.130.240.0/20', 'asn_country_code': 'US...","TDS-AS, US",a97557b5-b341-450d-ac71-cdd5658a7e03,OfficeActivityManager,2023-06-21 20:54:24+00:00,,MailItemsAccessed,NaN,...,None,,,,,,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",,,,,None,None,
2,69.130.255.13,69.130.255.13,US,United States,Wisconsin,Middleton,-89.5073,43.103,69.130.255.13,4181,69.130.255.13,69.130.240.0/20,US,arin,2003-03-25,"TDS-AS, US","[{'cidr': '69.128.0.0/14', 'handle': 'NET-69-128-0-0-1', 'name': 'NETBLK-TDSNET-BLK', 'startAddr...","{'asn': '4181', 'query': '69.130.255.13', 'asn_cidr': '69.130.240.0/20', 'asn_country_code': 'US...","TDS-AS, US",a97557b5-b341-450d-ac71-cdd5658a7e03,OfficeActivityManager,2023-06-21 20:54:24+00:00,,MailItemsAccessed,NaN,...,None,,,,,,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",,,,,None,None,
3,69.130.255.13,69.130.255.13,US,United States,Wisconsin,Middleton,-89.5073,43.103,69.130.255.13,4181,69.130.255.13,69.130.240.0/20,US,arin,2003-03-25,"TDS-AS, US","[{'cidr': '69.128.0.0/14', 'handle': 'NET-69-128-0-0-1', 'name': 'NETBLK-TDSNET-BLK', 'startAddr...","{'asn': '4181', 'query': '69.130.255.13', 'asn_cidr': '69.130.240.0/20', 'asn_country_code': 'US...","TDS-AS, US",a97557b5-b341-450d-ac71-cdd5658a7e03,OfficeActivityManager,2023-06-21 20:54:24+00:00,,MailItemsAccessed,NaN,...,None,,,,,,,,,,,,,,,,,"[{'Name': 'MailAccessType', 'Value': 'Bind'}, {'Name': 'IsThrottled', 'Value': 'False'}]",,,,,None,None,
4,69.130.255.13,69.130.255.13,US,United States,Wisconsin,Middleton,-89.5073,43.103,69.130.255.13,4181,69.130.255.13,69.130.240.0/20,US,arin,2003-03-25,"TDS-AS, US","[{'cidr': '69.128.0.0/14', 'handle': 'NET-69-128-0-0-1', 'name': 'NETBLK-TDSNET-BLK', 'startAddr...","{'asn': '4181', 'query': '69.130.255.13', 'asn_cidr': '69.130.240.0/20', 'asn_country_code': 'US...","TDS-AS, US",a97557b5-b341-450d-ac71-cdd5658a7e03,OfficeActivityManager,2023-06-21 20:58:17+00:00,,HardDelete,NaN,...,None,,,,,,,,,,,,,,,,,None,,,,,None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,NaN,2603:10b6:208:48a::21,US,United States,NaN,NaN,-97.8220,37.751,2603:10b6:208:48a::21,NA,2603:10b6:208:48a::21,NA,US,arin,2014-06-05,NA,"[{'cidr': '2603:1000::/24', 'handle': 'NET6-260

## Browse events with alternative sorting
You can pass a DataFrame to *result*`.view_events()` instead of an attribute name.

This means that you can apply sorting or filtering of the data before viewing it.
Here is an example sorting by IPAddress.

```python
acc_result.view_events(
    data=acc_result.azure_activity.sort_values("IPAddress"),
    summary_cols=["SourceSystem", "Operation", "IPAddress"]
)
```

## Additional properties and methods of the result object

These are static properties - usually DataFrames or visualizations. 
You can access each of these to see or manipulate the retrieved data.

To see help on the available attributes type:
```python
>>> help(acc_result)
```
To see the available methods type:
```python
>>> acc_result.list_methods()
```
> Note, for the AccountSummary notebooklet, the two main data retrieval methods are:<br>
> - run<br>
> - get_additional_data<br>
> There are several other methods that allow you to view individual plots
> or subsets of the data (such as alerts).

To view help on a specific method type:
```python
>>> help(acc_result.method_name)
```

To run a method
```python
acc_result.display_alert_timeline()
```

In [15]:
acc_result.list_methods()

["az_activity_timeline_by_ip - 'Display Azure activity timeline by IP address.'",
 "az_activity_timeline_by_operation - 'Display Azure activity timeline by operation.'",
 "az_activity_timeline_by_provider - 'Display Azure activity timeline by provider.'",
 "browse_accounts - 'Return the accounts browser/viewer.'",
 "browse_alerts - 'Return alert browser/viewer.'",
 "browse_bookmarks - 'Return bookmark browser/viewer.'",
 "display_alert_timeline - 'Display the alert timeline.'",
 "get_additional_data - 'Find additional data for the selected account.'",
 "get_geoip_map - 'Return Folium map of IP activity.'",
 "host_logon_timeline - 'Display IP address summary.'",
 "run - 'Return account activity summary.'",
 "show_ip_summary - 'Display Azure activity timeline by operation.'"]

---

## Viewing the Result class
You can view all of the data in the results class by "running" it in a cell

> Note: This produces a lot of output.<br>
> Due to the way Jupyter display Javascript objects the plots may
> appear out of order.

```ipython
  acc_result
```

Most of the properties of the results class are pandas DataFrames - 
you can use these directly for further analysis. Other property types
include entities and visualizations.

The DataFrames displayed by running the result object are truncated
to the first five rows.

You can also access individual data properties of the result as follows:
```ipython
  result.data_property
```

In [16]:
acc_result.data_properties()

['account_activity',
 'azure_activity',
 'azure_activity_summary',
 'ip_summary',
 'ip_all_data']

---

## Using Pivots to get more context information

You can run a pivot function on the account summary results
to get additional context on the data.

Here is an example of looking up Whois information for Azure IPAddress requests.

```python
whois_df = (
    acc_result                      # the results object
    .azure_activity[["IPAddress"]]  # the property and the column we want
    .drop_duplicates()              # drop duplicates
    .mp_pivot.run(                  # run the pivot function IpAddress 'whois' function
        IpAddress.util.whois, column="IPAddress" 
    )
)
whois_df
```

---

## Use other notebooklets and pivots functions to drill down on other entities

You may want to drill down on other entities in the Account data.
You can use methods of the IpAddress or Host entities, for example,
to look at these in more detail.

Run the ip_address_summary notebooklet pivot
```python
IpAddress = entities.IpAddress
ip_result = IpAddress.nblt.ip_address_summary("157.56.162.53")
```

View the TI results
```python
ip_result.browse_ti_results()
```

---

# More information:

## Notebooklets and Pivots
[Notebooklets](https://msticnb.readthedocs.io/en/latest/)

[Pivot functions](https://msticpy.readthedocs.io/en/latest/data_analysis/PivotFunctions.html)

## Notebook/MSTICPy configuration
[Getting Started](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/A%20Getting%20Started%20Guide%20For%20Azure%20Sentinel%20ML%20Notebooks.ipynb)
[MSTICPy Configuration guide](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)

[ConfigureNotebookEnvironment notebook](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)